# **MÓDULO 18 - Pratique**
# Regressão Linear

Agora que aprendemos como aplicar a regressão linear simples e múltipla, colocaremos em prática os conceitos vistos na aula.

Temos aqui uma base de imóveis para alugar, precisamos desenvolver um modelo de regressão linear múltipla para conseguir prever o preço de imóveis dadas as variáveis independentes do nosso modelo.

**Atenção! Esse é seu primeiro modelo, caso tenha dificuldade conte com a ajuda da tutoria**

Você notará que alguns códigos já estão presentes para facilitar a construção de vocês.

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [76]:
df = pd.read_csv("C:/Users/yurid/Downloads/ALUGUEL_MOD12.csv", delimiter=';')

df.head(10)

,Valor_Aluguel,Valor_Condominio,Metragem,N_Quartos,N_banheiros,N_Suites,N_Vagas
0,480,295,48,2,2,1,1
1,500,0,50,1,2,1,1
2,500,0,40,1,2,1,1
3,500,36,45,1,2,1,0
4,500,0,30,1,1,0,0
5,500,380,66,2,1,0,1
6,550,100,48,2,2,1,1
7,600,110,46,2,2,1,1
8,600,100,49,2,2,1,1
9,600,325,50,2,2,1,1


Legenda dos dados:

*   **Valor_Aluguel** : valor Total pago no aluguel

*   **Valor_Condominio** : Valor do Condomínio.

*   **Metragem** : Metragem do Apartamento.

*   **N_Quartos** : Número de Quartos do Imóvel.

*   **N_banheiros** : Número de banheiros.

*   **N_Suites** : Número de Suítes.

*   **N_Vagas** : Número de Vagas.

# 1 - Realize a primeira etapa de pré processamento dos dados.

A) Verifique os tipos de dados.


B) Verifique os dados faltantes, se houver dados faltantes faça a substituição ou remoção justificando sua escolha.

In [7]:
#Seu código aqui
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7203 entries, 0 to 7202
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Valor_Aluguel     7203 non-null   int64
 1   Valor_Condominio  7203 non-null   int64
 2   Metragem          7203 non-null   int64
 3   N_Quartos         7203 non-null   int64
 4   N_banheiros       7203 non-null   int64
 5   N_Suites          7203 non-null   int64
 6   N_Vagas           7203 non-null   int64
dtypes: int64(7)
memory usage: 394.0 KB


Valor_Aluguel       0
Valor_Condominio    0
Metragem            0
N_Quartos           0
N_banheiros         0
N_Suites            0
N_Vagas             0
dtype: int64

# 2 - Realize a segunda etapa de pré processamento dos dados.

A) Utilize a função describe para identificarmos outliers e verificarmos a distribuição dos dados.


B) Caso note uma variável que te pareça conter outliers realiza a análise e tratamento desses dados, justificando a escolha do método utilizado.

C) Realize a análise bivariada dos dados. Faça uso de pelo menos 3 gráficos e traga insights acerca do analisado.

In [8]:
print(df.describe())

       Valor_Aluguel  Valor_Condominio     Metragem    N_Quartos  N_banheiros  \
count    7203.000000       7203.000000  7203.000000  7203.000000  7203.000000   
mean     2966.596140        811.538109    88.506178     2.300153     2.095932   
std      2948.720385        796.564846    61.567505     0.826615     0.983812   
min       480.000000          0.000000    30.000000     1.000000     1.000000   
25%      1350.000000        395.000000    52.000000     2.000000     2.000000   
50%      2000.000000        592.000000    67.000000     2.000000     2.000000   
75%      3200.000000        980.000000   100.000000     3.000000     2.000000   
max     25000.000000       9500.000000   880.000000    10.000000     8.000000   

          N_Suites     N_Vagas  
count  7203.000000  7203.00000  
mean      1.016660     1.44176  
std       0.874204     0.86993  
min       0.000000     0.00000  
25%       1.000000     1.00000  
50%       1.000000     1.00000  
75%       1.000000     2.00000  
max   

In [91]:
fig = make_subplots(rows=1, cols = 7)
for i, coluna in enumerate(df.columns):
    fig.add_trace(go.Histogram(x=df[coluna],
                               name= coluna),
                  row=1,
                  col= i+1)
fig.update_layout(title_text = "Histograma dos dados",
                  legend_title = "Variável",
                  bargap = 0.2)
ddfig.show()

Conforme as distribuições acima, vemos que ambas as variáveis possuem outliers. Assim, vamos aplicar o método IQR para filtrar os dados de forma segura.

In [129]:
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
lim_inf = q1 -1.5*iqr
lim_sup = q3 +1.5*iqr

mascara = (df >= lim_inf) & (df <= lim_sup)

df_teste = df[mascara]


fig = make_subplots(rows=1, cols = 7)
for i, coluna in enumerate(df.columns):
    fig.add_trace(go.Histogram(x=df_teste[coluna],
                               name= coluna),
                  row=1,
                  col= i+1)
fig.update_layout(title_text = "Histograma dos dados após filtragem dos dados",
                  legend_title = "Variável",
                  bargap = 0.2)
fig.show()




Como as variáveis número de banheiros e suítes foram muito reduzidas pelo método IQR, vamos voltar os valores originais a elas. Depois, vamos filtrar os valores nulos advindos da retirada dos outliers. Em seguida, verificaremos as novas distribuições.

In [131]:
df_teste['N_banheiros'] = df['N_banheiros']
df_teste['N_Suites'] = df['N_Suites']
df_teste = df_teste.dropna()
df_teste.isnull().sum()


Valor_Aluguel       0
Valor_Condominio    0
Metragem            0
N_Quartos           0
N_banheiros         0
N_Suites            0
N_Vagas             0
dtype: int64

In [134]:
fig = make_subplots(rows=1, cols = 7)
for i, coluna in enumerate(df.columns):
    fig.add_trace(go.Histogram(x=df_teste[coluna],
                               name= coluna),
                  row=1,
                  col= i+1)
fig.update_layout(title_text = "Histograma dos dados após filtragem dos dados",
                  legend_title = "Variável",
                  bargap = 0.2)
fig.show()

Agora, conforme os gráficos acima, ainda que não normalmente distribuídas, temos melhores distribuições das variáveis. Neste momento, passamos para a análise bivariada, buscando responder 3 perguntas:

- 1) Como a quantidade de vagas e o valor do aluguel estão relacionados?
- 2) Como o número de quartos influencia no valor do aluguel?
- 3) Como o número de banheiros e o valor do aluguel estão relacionados?

In [167]:
medias_vagas = {}
medias_quartos = {}
medias_banheiros = {}
for valor in df_teste['N_Vagas'].unique():
    medias_vagas[valor] = df_teste['Valor_Aluguel'][df_teste['N_Vagas'] == valor].mean()

for valor in df_teste['N_Quartos'].unique():
    medias_quartos[valor] = df_teste['Valor_Aluguel'][df_teste['N_Quartos'] == valor].mean()

for valor in df_teste['N_banheiros'].unique():
    medias_banheiros[valor] = df_teste['Valor_Aluguel'][df_teste['N_banheiros'] == valor].mean()

df_vagas = pd.DataFrame(list(medias_vagas.items()), columns = ['Quantidade', 'Valor do Aluguel'])
df_quartos = pd.DataFrame(list(medias_quartos.items()), columns = ['Quantidade', 'Valor do Aluguel'])
df_banheiros = pd.DataFrame(list(medias_banheiros.items()), columns = ['Quantidade', 'Valor do Aluguel'])

fig = make_subplots(rows=1, cols=3)

fig.add_traces(go.Bar(x=df_vagas['Quantidade'],
                      y=df_vagas['Valor do Aluguel'],
                      name='Vagas',
                      marker_color = 'salmon'),
               rows=1,
               cols=1)
fig.add_traces(go.Bar(x=df_quartos['Quantidade'],
                      y=df_quartos['Valor do Aluguel'],
                      name='Quartos',
                      marker_color = 'orchid'),
               rows=1,
               cols =2)

fig.add_traces(go.Bar(x=df_banheiros['Quantidade'],
                      y=df_banheiros['Valor do Aluguel'],
                      name='Banheiros',
                      marker_color = 'lightblue'),
               rows=1,
               cols = 3)

fig.update_layout(legend_title = 'Variável')
fig.update_yaxes(title_text = 'Valor do Aluguel')
fig.update_xaxes(title_text = 'Quantidade')

Conforme os gráficos acima, vemos que, de forma geral, o aumento do número de vagas, quartos e banheiros encarece o valor do aluguel.

# 3 - Realize a terceira etapa de pré processamento dos dados.

A) Comece pela correlação, que sabemos ser uma parte importante para nosso pré processamento e análise. Plote o gráfico ou a tabela e indique as variáveis que te parecem mais "fortes" na correlação para nosso modelo.




In [180]:
correl = df.corr()
fig = go.Figure()
correl
fig.add_traces(go.Heatmap(z=correl,
                          y= correl.index,
                          x=correl.columns,
                          colorscale = 'pinkyl',
                          texttemplate = "%{z}"))


Dado o plot da correlação, temos que as maiores correlações estão em:
- Valor_Aluguel x Valor_Condominio (69,5%)
- Valor_Aluguel x Metragem (73%)
- Metragem x Valor_Condominio (80,5%)
- Metragem x N_Vagas (74,4%)
- N_Suites x N_banheiros (92%)
- N_Vagas x Valor_Condominio (69%)


B) Durante a aula, por nossa base ser pequena e demonstrativa não realizamos a separação de treino e teste, porém para as atividades do dia dia temos que fazer, nesse exercício separe treino e teste.


In [181]:
x = df.drop('Valor_Aluguel', axis=1) #Separando X - Todas variáveis exceto valor_aluguel
y = df['Valor_Aluguel'] #Separando Y (Apenas variavel valor_aluguel)
x_test, x_train, y_test, y_train = train_test_split(x, y, test_size = 0.25, random_state = 42)

# 3 - Treine um modelo de regressão Linear simples

A) Vamos utilizar apenas X_train e y_train para rodar um modelo de regressão linea simples e para isso usaremos apenas uma váriavel, a váriavel metragem.

In [187]:
X = x_train[['Metragem']] # Variável independente (características)
y = y_train  # Variável dependente (rótulo)

In [193]:
#Crie seu modelo aqui, usando LinearRegression e as bases de treino.
reg = LinearRegression().fit(X,y)
tabela = pd.DataFrame({'Coeficiente': reg.coef_,
                       'Intercepto': reg.intercept_,
                       'R²': reg.score(X,y)})
tabela

,Coeficiente,Intercepto,R²
0,35.691684,-199.403977,0.565597


B) Plote o intercept_ e coef_ e monte de forma extensa a equação da reta.

Nossa equação seria: y = 35.691684x - 199.403977

c) Calcule o R quadrado para o modelo de treinamento. Não esqueça de avaliar e trazer em formato de insight se esse resultado te parece bom ou não.

Dado o valor de R² igual a 0.565597, temos que o modelo não é adequado para fazer previsões, já que é esperado um valor maior que 0.8.

D) Plote o gráfico da reta de regressão encontrada e traga insights acerca da dispersão dos pontos e ajuste da reta.

In [210]:


fig = go.Figure()
fig.add_traces(go.Scatter(x=X['Metragem'],
                          y=y,
                          mode='markers',
                          marker_color = 'turquoise',
                          name = 'Dispersão'))
fig.add_traces(go.Scatter(x=X['Metragem'],
                          y=reg.predict(X),
                          marker_color = 'tomato',
                          name = 'Previsão'))
fig.update_layout(title_text = 'Regressão linear para valor do aluguel com base na metragem',
                  xaxis_title = 'Metragem',
                  yaxis_title = 'Aluguel')
fig.show()

Dado que os pontos estão dispersos ao redor da linha e o valor de R² baixo, temos que o modelo não é eficaz na previsão dos valores de aluguel.

E) Para finalizar vamos aplicar o modelo a base de teste. Essa etapa é nova, então agora vocês avaliaram como o modelo treinado se saiu com a base de testes.
Para isso altere no código abaixo o nome do seu modelo de regressão:

In [212]:
X_test = x_test[['Metragem']]  # Variável independente (características)
y_test = y_test  # Variável dependente (rótulo)

In [213]:
# Usando o modelo treinado para fazer previsões sobre os dados de teste
previsoes = reg.predict(X_test)

# Avaliando o desempenho do modelo usando métricas como o R²
r2 = reg.score(X_test, y_test)

print("Coeficiente de Determinação (R²) nos Dados de Teste:", r2)


Coeficiente de Determinação (R²) nos Dados de Teste: 0.5208979072026962


Se o valor do coeficiente de determinação (R²) para os dados de treinamento for melhor (ou seja, mais próximo de 1) do que o R² para os dados de teste, isso sugere que o modelo está superajustado aos dados de treinamento. Isso significa que o modelo pode estar se ajustando muito bem aos padrões específicos nos dados de treinamento, mas pode não generalizar bem para novos dados que não foram vistos durante o treinamento.

Por outro lado, se o R² para os dados de teste for melhor do que o R² para os dados de treinamento, isso pode ser indicativo de que o modelo está subajustado. Isso significa que o modelo não está se ajustando adequadamente aos padrões nos dados de treinamento e não está capturando a relação entre as variáveis independentes e dependentes de forma eficaz.

Idealmente, gostaríamos que o valor do R² fosse consistente entre os dados de treinamento e teste, indicando que o modelo é capaz de generalizar bem para novos dados. Se houver uma grande diferença entre os valores de R² para os dados de treinamento e teste, isso sugere que o modelo pode precisar de ajustes para melhorar sua capacidade de generalização.

F) Avalie com suas palavras o valor do r quadrado encontrado no treino e no teste.

Como o valor do R² de treino foi maior que o R² do teste, temos que o modelo está superajustado aos dados de treinamento, de modo que não generaliza bem para novas entradas.

# 4 - Aplicação do modelo de regressão linear multipla!

A) Vamos refazer os passos anteriores porém para regressão multipla, com todas variáveis dependentes. Comece separando a base treino e teste, dessa vez com todas variáveis para X.

Aqui é só refazer os passos do exercicio 3 porém ao invés de trazer para X apenas metragem, você deve trazer todas colunas (exceto a valor do aluguel).

In [219]:
X = df.drop('Valor_Aluguel', axis=1)
y = df['Valor_Aluguel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

B) Faça o modelo de regressão linear multipla aplicado só a base de treino.

In [220]:
reg = LinearRegression().fit(X,y)
tabela = pd.DataFrame({'Coeficiente': reg.coef_,
                       'Intercepto': reg.intercept_,
                       'R²': reg.score(X,y)})
tabela

,Coeficiente,Intercepto,R²
0,0.818904,345.21093,0.60761
1,20.996401,345.21093,0.60761
2,-651.151752,345.21093,0.60761
3,232.192261,345.21093,0.60761
4,300.529175,345.21093,0.60761
5,557.690253,345.21093,0.60761


C) Traga o valor do R quadrado e avalie o valor encontrado.

Dado R² = 0.6076, temos um valor melhor que o exercício anterior, mais ainda baixo.

D) Para finalizar aplique o modelo a base de teste e traga o r quadrado de teste.
Dica: Você pode usar os códigos do exercício anterior.

In [221]:
# Usando o modelo treinado para fazer previsões sobre os dados de teste
previsoes = reg.predict(X_test)

# Avaliando o desempenho do modelo usando métricas como o R²
r2 = reg.score(X_test, y_test)

print("Coeficiente de Determinação (R²) nos Dados de Teste:", r2)


Coeficiente de Determinação (R²) nos Dados de Teste: 0.6350572928561846


E) Compare os r quadrados encontrados pela regressão linear e pela regressão múltipla. Qual modelo te parece melhor? Por qual motivo acredita que isso ocorreu?

Dado que o R² da regressão múltipla foi maior que o da regressão linear, tanto no teste, quanto no treino, temos que a regressão múltipla s eaplica melhor. No entanto, enquanto a regressão linear apresentou um modelo superajustado aos dados de treino, a regressão múltipla apresentou o contrário, já que o R² do teste foi maior que o de treino. 